In [36]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os
import torch.nn.functional as F


In [37]:
# ✅ Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [38]:
# ✅ Correct CNN model architecture matching your trained model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Output: 32x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 64x64x64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 64x32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# 128x32x32
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 128x16x16

            nn.Flatten(),                                # 128*16*16 = 32768
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)
        )

    def forward(self, x):
        return self.net(x)

# ✅ Load the model
model = CNNModel().to(DEVICE)
model.load_state_dict(torch.load("fingerprint_cnn_with_val.pth", map_location=DEVICE))
model.eval()
print("✅ Model loaded successfully!")

✅ Model loaded successfully!


In [39]:
# ✅ Class labels
class_names = ['Real', 'Altered-Easy', 'Altered-Medium', 'Altered-Hard']

In [40]:
# 🔄 Transformations (same as training)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])



In [41]:
# 🔍 Prediction function
def predict_bmp_image(image_path):
    image = Image.open(image_path).convert("L")
    image = transform(image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(image)
        probs = F.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        predicted_class = class_names[pred_idx]
        confidence = probs[0][pred_idx].item()

    print(f"🖼️ Image: {os.path.basename(image_path)} ➜ Predicted: {predicted_class} ({confidence * 100:.2f}%)")
    return predicted_class, confidence

In [42]:
test_image_path = r"E:\Biometrics\balanced_dataset\Real\1__M_Left_little_finger.BMP"
predict_bmp_image(test_image_path)

🖼️ Image: 1__M_Left_little_finger.BMP ➜ Predicted: Altered-Hard (100.00%)


('Altered-Hard', 0.9999686479568481)